# Linear Algebra

numpy is Python's most important package for linear algebra.  It defines vectors, matrices, and their operations, in a very efficient implementation.

In [ ]:
import numpy as np
import time

The most important data type in numpy is ``ndarray``.  These are general matrices.  A special subclass is ``matrix``, which creates an intuitive notation for 2D matrices.  It's otherwise not very helpful and generally not used.

A vector is, of course, nothing but a special type of matrix.

In [ ]:
a = np.array([1,2,3])
print (a, a.shape)

The above is not really a vector, of course, but rather the transpose of it.  We can, of course, transpose a, but it makes no real difference for the internal representation of ``a``:

In [ ]:
a = np.transpose(a)
print (a, a.shape)

By the way, the nice thing of numpy is that it is under-the-hood implemented in C.  That makes it fast.  Much faster.  Compare the following.

In [ ]:
#compare speeds
array = np.random.rand(100)
print(array)

The dot product of a with itself is the Euclidean length squared of the vector that is represented by ``a``:

In [ ]:
%%timeit
s = 0
for i in range(0,array.shape[0]):
    s += array[i]*array[i] 


This is known as the L2 norm (squared).  But numpy has an efficient implementation, which is not only easier to write but also faster:

In [ ]:
%%timeit
s = np.dot(array, array)


The same works for matrices:

In [ ]:
A = np.array([[1, 2], [3, 4], [5, 6]])
print (A, A.shape)
B = np.array([[-1, 1, 2], [1, 2, -2]])
print (B, B.shape)

How do we multiply two matrices?

In [ ]:
def formult(M1, M2): 
    if (M1.shape[1] != M2.shape[0]):
        raise IndexError('matrices don\'t match for multiplication')
    C = np.zeros((M1.shape[0], M2.shape[1]))
    for i in range(0,M1.shape[0]):
        for j in range(0,M2.shape[1]):
            for k in range(0,M1.shape[1]):
                C[i,j] += M1[i,k] * M2[k,j]
    return C

In [ ]:
%%timeit
formult(A, B)

but for loops are not efficient in python.  There usually is a simpler way to do it.  Of course, numpy has this buit in:
    

In [ ]:
%%timeit
np.dot(A, B)

and it is much faster!

In [ ]:
t1 = time.time()
for i in range(0,10000):
    C1 = formult(A, B)
t2 = time.time()
for i in range(0,10000):
    C2 = np.dot(A, B)
t3 = time.time()
print(C1, t2-t1)
print(C2, t3-t2)
print("the for loop is", (t2-t1)/(t3-t2), "times slower")

The identity matrix is quickly built in numpy:

In [ ]:
np.identity(3)

other special matrices:

In [ ]:
print(np.zeros((3,4)))
print(np.ones((2,3)))

Beware of how we multiply matrices with matrices (and with vectors etc.)  Which of these will work?


In [ ]:
np.dot(A, B)

In [ ]:
np.dot(A, a)

The outer product is the "opposite" of the inner product.  While the inner product of vectors $a$ and $b$ is $a \cdot b = a^Tb$, the outer product $a \otimes b = a b^T$.

In [ ]:
np.outer(A, a)

The outer product of two vectors is a matrix!

In [ ]:
np.outer(a, a)

So I can compute the dot ("inner") product of matrix A and B, since A has dimensions $n\times m$ and B $m\times p$.  But $A.A = A^T A$ does not work!  For that I have to switch rows and columns of A, called the transpose.
The transpose of a matrix is quickly found:

In [ ]:
print(A, A.shape)
print(np.transpose(A), np.transpose(A).shape)
print(np.dot(A,A))    # goes horribly wrong, why?
print(np.dot(A,A.T))  # A.T is a more readable way of getting transpose

Determinant
-----------
The determinant of a matrix is the volume of the transformation of that matrix.  For the matrix [[a,b] [c,d]]:
<img src="det-2d.png" width="300">
For a $3\times3$ matrix:
<img src="det-3d.png" width="300">
Please compute the matrix of a $1\times1$ matrix, of a $2\times2$ matrix, and...

In [ ]:
C = np.ones((1,1)) * 10
print(C)
np.linalg.det(C)

In [ ]:
C = np.empty((2,2))
C = [[1, 2], [3, 4]]
print(np.linalg.det(C))  # 1 * 3 - 2 * 4

In [ ]:
C = np.ones((2,3))
...


what happened?

# matrix properties
A symmetric matrix is one which equals its transpose.  Or you can say, $B_{ij} = B_{ji}$.  How do we create a symmetric matrix in Python?

Let's first create a random matrix.  And its transpose.  Then (assignment) create a symmetric one.

In [ ]:
B = np.random.randint(-10,10,size=(2,2))
print(B)
print(B.T)
# now create a symmetric matrix out of B and B.T
S = np.matmul(B.T, B) # or B.T.dot(B)
S = B.T.dot(B)
print(S - S.T)

A _positive definite_ matrix, defined as a square matrix with $x^T A x > 0$ for all $x$, has all its eigenvalues positive.  Also, the diagonal elements are positive.  But... what are eigenvalues?

Eigenvalues and -vectors
-------------
One can imagine eigenvectors to represent the "invariant" directions that a matrix represents. Eigenvectors are only defined for square matrices.  An $n\times n$ matrix can be seen as transforming vectors from one $n$-dimensional space to another.  

If $v$ is an eigenvector, $\lambda$ is the corresponding eigenvalue such that $A v = \lambda v$.

In the below figure the blue arrow represents an eigenvector (which does not change direction) of the matrix that "skews" the picture.  What is the eigenvalue of the blue eigenvector?

<img src="eigen.png">

In [ ]:
D = np.array([[1,2],[2,1]])
np.linalg.eig(D)

The condition of a matrix is the largest divided by the smallest ev.
If this number is very large (e.g. $> 10^5$ or so), the matrix is said to be badly conditioned.  It "magnifies" one dimension much, much more than another one!  If it is infinity, the matrix is singular: it has dependent rows.  

In [ ]:
np.linalg.cond(D)

In [ ]:
E = np.array([[1,2,3],[1,2,3],[2,4,2],[-5,3,5]])
print(np.linalg.cond(E))
E = np.array([[1,2],[0,0.00000001]])
print(np.linalg.cond(E))

The trace of a matrix is the sum of its eigenvalues.  The determinant of a matrix is the product of its eigenvalues.


# Solving a set of equations
-------------------------
Suppose we want to solve a set of linear equations $X w = z$.  This is linear regression: how do we find the best values of the parameters $w$ such that, when multiplied with the data $X$, the desired output $z$ is given?

In the mathematically clean situation, we have as many equations (rows in $X$) as parameters (rows in $w$).  In that case, $X$ is a square matrix, and the solution is found by $w = X^{-1} z$. 

In [ ]:
X = np.random.random((2,2))
print(X, np.linalg.cond(X))
print(np.dot(X, np.linalg.inv(X))) #X . X^{-1} should be the indentity matrix
w = np.dot(np.linalg.inv(X), np.random.random((2,1)))
print(w)

But if $X$ gets large... inversion is no longer a solution.

In [ ]:
X = np.random.random((2000,2000)) # don't run with 20000 x 20000, it is 400MB numbers!
print(np.dot(X, np.linalg.inv(X))) #X . X^{-1} should be the indentity matrix

Of course, this is also a problem if (a) $X$ is singular or nearly singular; and (b) when $X$ is not square.


In [ ]:
E = [[1,2.],[1,2.000000000000001]]  # note I forgot np.array().  Why does it work?
print(np.linalg.inv(E))
print(np.dot(np.linalg.inv(E) , E))

In [ ]:
# create a random matrix
X = np.random.random((5,5))
# let's say it is singular
X[2,:] = 0
# compute SVD
U,s,V = np.linalg.svd(X)
inverse = np.dot(np.dot(V.T, np.linalg.pinv(np.diag(s))), U.T)
print(np.dot(inverse, X))

The Moore-Penrose pseudoinverse in numpy does exactly the same:

In [ ]:
print(np.dot(np.linalg.pinv(X), X))

and it also works for non-square matrices

In [ ]:
X = np.random.random((3,4))
print(np.dot(np.linalg.pinv(X), X))

## Cholesky decomposition

Often, we want to find the root of a matrix: given a matrix $C$ we want to find a lower triangular matrix $L$ with $L L^T = C$. For this, the Cholesky decomposition can be used.

In [ ]:
X = np.random.random((2, 2))
C = np.dot(X.T, X)
print(C)

In [ ]:
L = np.linalg.cholesky(C)
print(L)
print(np.dot(L, L.T))

This can only be used for matrices which are positive definite, i.e. $\forall x: x^TCx > 0$. This is similar to the fact that we can only find the square root of positive numbers. In fact, it is a generalisation.

# reading material
http://cs229.stanford.edu/section/cs229-linalg.pdf

http://matrizen-rechner.de/ 

Gene Golub and Charles van Loan, Matrix Computations, John Hopkins University Press